## Update example to STAC1.0 

In [1]:
!pip install git+https://github.com/scottyhq/sentinel1.git@redux1.0

  Cloning https://github.com/scottyhq/sentinel1.git (to revision redux1.0) to /tmp/pip-req-build-zwa6ye3y
  Running command git clone -q https://github.com/scottyhq/sentinel1.git /tmp/pip-req-build-zwa6ye3y
  Running command git checkout -b redux1.0 --track origin/redux1.0
  Switched to a new branch 'redux1.0'
  Branch 'redux1.0' set up to track remote branch 'redux1.0' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for stactools-sentinel1: filename=stactools_sentinel1-0.1.5-py3-none-any.whl size=10542 sha256=2338d25bf740d811377b858cf65c2e4ce64bf167c3e91c6b34f979e7c6cc53f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-oakqz159/wheels/9e/4d/22/65209ca2f21ac55f5b5e1ddf8437caa856710b64baa2a65fa0
Successfully built stactools-sentinel1


In [2]:
import pystac
from pystac import Catalog, Collection, CatalogType
from pystac.layout import TemplateLayoutStrategy
from stactools.sentinel1.stac import create_item, create_collection
import concurrent.futures
%load_ext autoreload
%autoreload 2

In [3]:
# Load previously saved paths 
with open('paths.txt') as f:
    paths = [x.rstrip() for x in f.readlines()]

In [4]:
paths[:4]

['sentinel-s1-rtc-indigo/tiles/RTC/1/IW/10/U/CU/2017/S1A_20170101_10UCU_ASC',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/10/U/CU/2017/S1A_20170125_10UCU_ASC',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/10/U/CU/2017/S1B_20170102_10UCU_ASC',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/10/U/CU/2018/S1B_20180102_10UCU_ASC']

In [5]:
def s3_to_http(s3path, region='us-west-2'):
    s3prefix = 'sentinel-s1-rtc-indigo'
    newprefix = f'https://sentinel-s1-rtc-indigo.s3.{region}.amazonaws.com'
    http = s3path.replace(s3prefix, newprefix)
    #print(http)
    return http

In [6]:
item = create_item(s3_to_http(paths[0]))
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'S1A_20170101_10UCU_ASC',
 'properties': {'providers': [{'name': 'ESA',
    'roles': ['producer', 'processor', 'licensor'],
    'url': 'https://earth.esa.int/web/guest/home'},
   {'name': 'Indigo Ag Inc.',
    'roles': ['processor', 'licensor'],
    'url': 'https://registry.opendata.aws/sentinel-1-rtc-indigo'}],
  'constellation': 'sentinel-1',
  'platform': 'sentinel-1a',
  'instruments': ['c-sar'],
  'gsd': 20,
  'start_datetime': '2017-01-01T02:10:07Z',
  'end_datetime': '2017-01-01T02:10:36Z',
  'created': '2021-06-26T02:21:19.441253Z',
  'mgrs:utm_zone': '10',
  'mgrs:latitude_band': 'U',
  'mgrs:grid_square': 'CU',
  'sentinel:mgrs': '10UCU',
  'sentinel:product_ids': ['S1A_IW_GRDH_1SDV_20170101T021007_20170101T021036_014632_017CAA_D7AD'],
  'sentinel:data_coverage': '66.306',
  'sar:frequency_band': 'C',
  'sar:center_frequency': 5.405,
  'sar:observation_direction': 'right',
  'sar:instrument_mode': 'IW',
  'sar:product_type'

In [7]:
%%time 
# Generate all the items serially (could easily parallelize)
#items = [create_item(s3_to_http(f'{i}')) for i in paths]
# CPU times: user 1min 52s, sys: 17.2 s, total: 2min 9s
# Wall time: 4min 19s

httpPaths = [s3_to_http(f'{i}') for i in paths]
with concurrent.futures.ThreadPoolExecutor() as executor:
    items = executor.map(create_item, httpPaths)

CPU times: user 2min 12s, sys: 20.3 s, total: 2min 32s
Wall time: 1min 4s


In [8]:
collection = create_collection()
collection.add_items(items)
#collection.describe()

* <Collection id=sentinel1-rtc-aws>
  * <Item id=S1A_20170101_10UCU_ASC>
  * <Item id=S1A_20170125_10UCU_ASC>
  * <Item id=S1B_20170102_10UCU_ASC>
  * <Item id=S1B_20180102_10UCU_ASC>
  * <Item id=S1B_20180107_10UCU_ASC>
  * <Item id=S1B_20180109_10UCU_ASC>
  * <Item id=S1B_20190102_10UCU_ASC>
  * <Item id=S1B_20190104_10UCU_ASC>
  * <Item id=S1B_20190109_10UCU_ASC>
  * <Item id=S1B_20200109_10UCU_ASC>
  * <Item id=S1B_20200111_10UCU_ASC>
  * <Item id=S1B_20200121_10UCU_ASC>
  * <Item id=S1B_20210101_10UCU_DSC>
  * <Item id=S1B_20210105_10UCU_ASC>
  * <Item id=S1B_20210108_10UCU_DSC>
  * <Item id=S1B_20170102_10UFV_ASC>
  * <Item id=S1B_20170104_10UFV_ASC>
  * <Item id=S1B_20170121_10UFV_ASC>
  * <Item id=S1B_20180104_10UFV_ASC>
  * <Item id=S1B_20180109_10UFV_ASC>
  * <Item id=S1B_20180111_10UFV_ASC>
  * <Item id=S1B_20190104_10UFV_ASC>
  * <Item id=S1B_20190106_10UFV_ASC>
  * <Item id=S1B_20190111_10UFV_ASC>
  * <Item id=S1B_20200101_10UFV_ASC>
  * <Item id=S1B_20200106_10UFV_ASC>
  

In [9]:
# Save the catalog in current directory
template = '${sentinel:mgrs}/${year}'
collection.generate_subcatalogs(template=template)
collection.describe()

* <Collection id=sentinel1-rtc-aws>
    * <Catalog id=10UCU>
        * <Catalog id=2017>
          * <Item id=S1A_20170101_10UCU_ASC>
          * <Item id=S1A_20170125_10UCU_ASC>
          * <Item id=S1B_20170102_10UCU_ASC>
        * <Catalog id=2018>
          * <Item id=S1B_20180102_10UCU_ASC>
          * <Item id=S1B_20180107_10UCU_ASC>
          * <Item id=S1B_20180109_10UCU_ASC>
        * <Catalog id=2019>
          * <Item id=S1B_20190102_10UCU_ASC>
          * <Item id=S1B_20190104_10UCU_ASC>
          * <Item id=S1B_20190109_10UCU_ASC>
        * <Catalog id=2020>
          * <Item id=S1B_20200109_10UCU_ASC>
          * <Item id=S1B_20200111_10UCU_ASC>
          * <Item id=S1B_20200121_10UCU_ASC>
        * <Catalog id=2021>
          * <Item id=S1B_20210101_10UCU_DSC>
          * <Item id=S1B_20210105_10UCU_ASC>
          * <Item id=S1B_20210108_10UCU_DSC>
    * <Catalog id=10UFV>
        * <Catalog id=2017>
          * <Item id=S1B_20170102_10UFV_ASC>
          * <Item id=S1B_2

In [10]:
collection.normalize_hrefs('./')
collection.validate()

['https://schemas.stacspec.org/v1.0.0/collection-spec/json-schema/collection.json',
 'https://stac-extensions.github.io/sar/v1.0.0/schema.json',
 'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
 'https://stac-extensions.github.io/projection/v1.0.0/schema.json',
 'https://stac-extensions.github.io/raster/v1.0.0/schema.json',
 'https://stac-extensions.github.io/mgrs/v1.0.0/schema.json']

In [11]:
collection.save(catalog_type=pystac.CatalogType.RELATIVE_PUBLISHED)